In [6]:
import torch
import torchvision
import torch.nn as nn
import torch.nn.functional as F
from torchvision import datasets, models, transforms
from dataset_train import Dataset
from torch.utils.data import DataLoader

import os
import cv2
import numpy as np

from time import time
from visdom import Visdom
import copy

# Define important global variables 
# Change 'pretrained' variable to decide to get the pretrained weight or not
# Change 'feature_extract' variable to decide to keep all layers to update or freeze a part of the model
num_classes = 7
batch_size = 32
num_epochs = 15
feature_extract = True
pretrained = True
num_epochs = 1000
input_size = 224

# Use Visdom to visualise the training progress
viz = Visdom()
step_list = [0]
win = viz.line(X=np.array([0]), Y=np.array([1.0]), opts=dict(title='loss'))


Setting up a new session...


In [7]:
# This function is used to tell the optimizer not to update params in the freeze layer 
# Only use in freeze mode
def set_parameter_requires_grad(model, feature_extracting):
    if feature_extracting:
        for param in model.parameters():
            param.requires_grad = False

In [8]:
def train_model(model, dataloaders, criterion, optimizer, num_epochs):
    since = time()

    best_model_wts = copy.deepcopy(model.state_dict())
    best_acc = 1000.0

    count = 0
    for epoch in range(num_epochs):
        print('Epoch {}/{}'.format(epoch, num_epochs - 1))
        print('-' * 10)

        model.train()  # Set model to training mode
        
        running_loss = 0.0
        running_corrects = 0

        count += 1
        # Iterate over data.
        for step, (inputs, labels) in enumerate(dataloaders):
            inputs = inputs.cuda()
            labels = labels.cuda()
            

            # zero the parameter gradients
            optimizer.zero_grad()

            # forward pass and calculate loss
            outputs = model(inputs)
            loss = criterion(outputs, labels)
            _, preds = torch.max(outputs, 1)

            # Back prop and update params
            loss.backward()
            optimizer.step()

            # statistics
            running_loss += loss.item() * inputs.size(0)
            running_corrects += torch.sum(preds == labels)

            if step % 1 == 0:
                step_list.append(step_list[-1] + 1)
                viz.line(X=np.array([step_list[-1]]), Y=np.array([loss.item()]), win=win, update='append')
                
            if step % 100 == 0:    
                print('     step:{}, loss:{:.3f}, time:{:.3f} min'
                    .format(step, loss.item(), (time() - since) / 60))

        # deep copy the model
        if loss.item() < best_acc:
            count = 0
            best_acc = loss.item()
            best_model_wts = copy.deepcopy(model.state_dict())
            torch.save(model.state_dict(), './model/net{}-{:.3f}.pth'.format(epoch, loss))

        if count == 10:
            break
        print()

    time_elapsed = time() - since
    print('Training complete in {:.0f}m {:.0f}s'.format(time_elapsed // 60, time_elapsed % 60))
    print('Best val Acc: {:4f}'.format(best_acc))

    # load best model weights
    model.load_state_dict(best_model_wts)
    return model

In [9]:
BASE_DIR = '/home/jun/Github/BoVW/dataset/'
impath = os.listdir(BASE_DIR + 'train')

# Load data
train_ds = Dataset(BASE_DIR)
train_dl = DataLoader(train_ds, batch_size=batch_size, shuffle=True)

# Init model and customize from the pretrained version in Pytorch
model = models.vgg11_bn(pretrained=pretrained).cuda()
set_parameter_requires_grad(model, feature_extract)
num_ftrs = model.classifier[6].in_features
model.classifier[6] = nn.Linear(num_ftrs,num_classes)
model =  torch.nn.DataParallel(model).cuda()

loss_fn = torch.nn.CrossEntropyLoss()

# Choose params to update 
params_to_update = model.parameters()
if feature_extract:
    params_to_update = []
    for name,param in model.named_parameters():
        if param.requires_grad == True:
            params_to_update.append(param)
            
optimizer = torch.optim.SGD(params_to_update, lr=0.001, momentum=0.9)

['sea', 'green', 'city', 'house_building', 'face', 'house_indoor', 'office']


In [10]:
torch.cuda.empty_cache()

model_ft = train_model(model, train_dl, loss_fn, optimizer, num_epochs=num_epochs)

Epoch 0/999
----------
     step:0, loss:1.903, time:0.010 min

Epoch 1/999
----------
     step:0, loss:0.698, time:0.207 min

Epoch 2/999
----------
     step:0, loss:0.275, time:0.403 min

Epoch 3/999
----------
     step:0, loss:0.369, time:0.572 min

Epoch 4/999
----------
     step:0, loss:0.168, time:0.767 min

Epoch 5/999
----------
     step:0, loss:0.122, time:0.941 min

Epoch 6/999
----------
     step:0, loss:0.167, time:1.109 min

Epoch 7/999
----------
     step:0, loss:0.089, time:1.323 min

Epoch 8/999
----------
     step:0, loss:0.145, time:1.513 min

Epoch 9/999
----------
     step:0, loss:0.129, time:1.693 min

Epoch 10/999
----------
     step:0, loss:0.087, time:1.878 min

Epoch 11/999
----------
     step:0, loss:0.137, time:2.063 min

Epoch 12/999
----------
     step:0, loss:0.101, time:2.242 min

Epoch 13/999
----------
     step:0, loss:0.068, time:2.417 min

Epoch 14/999
----------
     step:0, loss:0.104, time:2.591 min

Epoch 15/999
----------
     step:0

In [16]:
test_img = cv2.imread(BASE_DIR + 'train/city/city-001.jpg')
test_img = cv2.resize(test_img, (input_size,input_size), interpolation=cv2.INTER_CUBIC)
test_img = np.moveaxis(test_img,2,0)
test_img = torch.FloatTensor(test_img).cuda().unsqueeze(0)
print(test_img.shape)

outputs = model(test_img)
print(outputs.shape)
print(outputs)

torch.Size([1, 3, 224, 224])
torch.Size([1, 10])
tensor([[ 3.7469,  4.1745,  3.6741,  0.9418,  3.2718, -0.5444,  4.6354, -5.8447,
         -4.9286, -6.8093]], device='cuda:0', grad_fn=<AddmmBackward0>)
